Next steps: Custom softmax & other improvements, MultiplexNet

Check out Decision Transformer: Reinforcement Learning via Sequence Modeling

In [1]:
import numpy as np
import pandas as pd

import torch as th
from torch.optim import Adam
from torcheval.metrics import MeanSquaredError
from training_loop import TrainingLoop, SimpleTrainingStep
from training_loop.callbacks import EarlyStopping, TensorBoardLogger
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from envs import BlendEnv, reconstruct_dict

import datetime
from models import IL_MLP_simple, IL_RNN_simple
import os

In [26]:
now = datetime.datetime.now().strftime('%m%d-%H%M')
modelname = f"MLP_{now}"

In [27]:
cols_obs = [
    'sources_s1', 'sources_s2',
    'blenders_j1', 'blenders_j2', 'blenders_j3', 'blenders_j4',
    'demands_p1', 'demands_p2',
    'properties_j1_q1', 'properties_j2_q1', 'properties_j3_q1', 'properties_j4_q1',
    'sources_avail_next_0_s1', 'sources_avail_next_0_s2', 'demands_avail_next_0_p1', 'demands_avail_next_0_p2',
    'sources_avail_next_1_s1', 'sources_avail_next_1_s2', 'demands_avail_next_1_p1', 'demands_avail_next_1_p2',
    'sources_avail_next_2_s1', 'sources_avail_next_2_s2', 'demands_avail_next_2_p1', 'demands_avail_next_2_p2',
    'sources_avail_next_3_s1', 'sources_avail_next_3_s2', 'demands_avail_next_3_p1', 'demands_avail_next_3_p2',
    'sources_avail_next_4_s1', 'sources_avail_next_4_s2', 'demands_avail_next_4_p1', 'demands_avail_next_4_p2',
    'sources_avail_next_5_s1', 'sources_avail_next_5_s2', 'demands_avail_next_5_p1', 'demands_avail_next_5_p2',
    't'
]
cols_act = [
    'source_blend_s1_j1', 'source_blend_s1_j2', 'source_blend_s1_j3', 'source_blend_s1_j4',
    'source_blend_s2_j1', 'source_blend_s2_j2', 'source_blend_s2_j3', 'source_blend_s2_j4',
    'blend_demand_j1_p1', 'blend_demand_j1_p2',
    'blend_demand_j2_p1', 'blend_demand_j2_p2',
    'blend_demand_j3_p1', 'blend_demand_j3_p2',
    'blend_demand_j4_p1', 'blend_demand_j4_p2',
    'tau_s1', 'tau_s2',
    'delta_p1', 'delta_p2'
]

In [28]:
X, y = pd.DataFrame([], columns=cols_obs), pd.DataFrame([], columns=cols_act)

for file in os.listdir("./data/simple"):
    df = pd.read_csv(os.path.join("./data/simple", file))
    if "OBS" in file:
        X = pd.concat([X, df])
    elif "ACT" in file:
        y = pd.concat([y, df])

X = X.drop("Unnamed: 0", axis=1)
y = y.drop("Unnamed: 0", axis=1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [29]:
# Normalization to [0, 1]
X_min = X.min()
y_min = y.min()
X_max = X.max()
y_max = y.max()

def norm_X(X):
    return (X - X_min) / (X_max - X_min)
def denorm_X(X):
    return X * (X_max - X_min) + X_min
def norm_y(y):
    return (y - y_min) / (y_max - y_min)
def denorm_y(y):
    return y * (y_max - y_min) + y_min

X_train = norm_X(X_train)
X_val = norm_X(X_val)
X_test = norm_X(X_test)
y_train = norm_y(y_train)
y_val = norm_y(y_val)
y_test = norm_y(y_test)

In [6]:
class MyData(Dataset):
    def __init__(self, df_X: pd.DataFrame, df_y: pd.DataFrame) -> None:
        super().__init__()
        self.df_X = df_X
        self.df_y = df_y
        
    def __getitem__(self, index):
        return(th.Tensor(self.df_X.iloc[index]), th.Tensor(self.df_y.iloc[index]))
    
    def __len__(self):
        return(self.df_X.shape[0])

In [7]:
train_ds = MyData(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=16)
val_ds = MyData(X_val, y_val)
val_dl = DataLoader(val_ds, batch_size=16)
test_ds = MyData(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=16)
len(train_ds)

1155

In [8]:
model = IL_RNN_simple()

In [9]:
loop = TrainingLoop(
    model,
    step=SimpleTrainingStep(
        optimizer_fn = lambda params: Adam(params, lr=1e-4),
        loss = th.nn.MSELoss(),
        # metrics = ('MSE', MeanSquaredError()),
    ),
    device='cpu'
)
loop.fit(
    train_dl,
    val_dl,
    epochs=100,
    callbacks=[
        EarlyStopping(monitor='val_loss', mode='min', patience=20),
        TensorBoardLogger(f"./logs/simple/imit/{modelname}")
    ],
    
)

Epoch 1/100:   0%|          | 0/82 [00:00<?, ?it/s]

torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([1

Epoch 2/100:   0%|          | 0/82 [00:00<?, ?it/s]

torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([1

Epoch 3/100:   0%|          | 0/82 [00:00<?, ?it/s]

...

Epoch 61/100:   0%|          | 0/82 [00:00<?, ?it/s]

torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([1

Epoch 62/100:   0%|          | 0/82 [00:00<?, ?it/s]

torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([16, 37])
torch.Size([1

(                 loss  batch  epoch
 epoch batch                        
 1      1     0.032745      1      1
        2     0.030075      2      1
        3     0.035197      3      1
        4     0.039809      4      1
        5     0.038911      5      1
 ...               ...    ...    ...
 62     70    0.032841     70     62
        71    0.032967     71     62
        72    0.033103     72     62
        73    0.033018     73     62
       -1     0.033018     -1     62
 
 [4588 rows x 3 columns],
                      val_loss  val_batch  val_epoch
 val_epoch val_batch                                
 1          1         0.046227          1          1
            2         0.050270          2          1
            3         0.057505          3          1
            4         0.059332          4          1
            5         0.060199          5          1
 ...                       ...        ...        ...
 62         6         0.049116          6         62
            7 

In [16]:
th.save(model.state_dict(), f"weights/{modelname}.pth")

In [10]:
connections = {
    "source_blend": {
        "s1": ["j1", "j2", "j3", "j4"],
        "s2": ["j1", "j2", "j3", "j4"]
    },
    "blend_blend": {"j1": [], "j2": [], "j3": [], "j4": []},
    "blend_demand": {"j1": ["p1", "p2"],
        "j2": ["p1", "p2"],
        "j3": ["p1", "p2"],
        "j4": ["p1", "p2"]
    }
}

action_sample = {
    'source_blend':{
        's1': {'j1':1, 'j2':1, 'j3':1, 'j4':0}, # From s1 to b1, from s1 to b2 etc...
        's2': {'j1':1, 'j2':1, 'j3':0, 'j4':1},
    },
    
    'blend_blend':{},
    
    'blend_demand':{
        'j1': {'p1':1, 'p2':0},
        'j2': {'p1':1, 'p2':2},
        'j3': {'p1':1, 'p2':2},
        'j4': {'p1':1, 'p2':2}
    },
    
    "tau": {"s1": 10, "s2": 10},
    
    "delta": {"p1": 0, "p2": 0}
}

In [11]:
M=P=Q=B=Z=D=0

In [30]:
env = BlendEnv(connections = connections, action_sample=action_sample, M=M, P=P, Q=Q, B=B, Z=Z, D=D)

In [31]:
th.Tensor(X_train.iloc[0].to_numpy(dtype=float)).shape

torch.Size([37])

In [32]:
env = env
with th.no_grad():
    obs = env.reset()
    obs, obs_dict = obs
    for k in range(env.T):
        obs = th.Tensor(norm_X(obs)).unsqueeze(0)
        
        action = np.array(model.forward(obs).squeeze(0))
        action = denorm_y(action)
        
        print(action.shape)
        print("\n\n   ",reconstruct_dict(action, env.mapping_act))
        
        obs, reward, done, term, _ = env.step(action)
        dobs = reconstruct_dict(obs, env.mapping_obs)
        
        print("\n    >>     ",dobs["sources"], dobs["blenders"], dobs["demands"])
        print("   " ,reward)

torch.Size([1, 37])
(20,)


    {'source_blend': {'s1': {'j1': 1.2619007751345634, 'j2': -9.696550939104586e-08, 'j3': 0.0, 'j4': 1.1671499853100238}, 's2': {'j1': 1.0076270252466202, 'j2': 0.6560637664903267, 'j3': 0.0, 'j4': 0.2169148251414299}}, 'blend_demand': {'j1': {'p1': 0.0, 'p2': 0.0}, 'j2': {'p1': -9.607639056656139e-08, 'p2': 0.0}, 'j3': {'p1': 0.0, 'p2': 0.0}, 'j4': {'p1': 0.0, 'p2': 0.0}}, 'tau': {'s1': 3.998083770275116, 's2': 0.0}, 'delta': {'p1': 0.0, 'p2': 0.0}}

    >>      {'s1': 1.5690331, 's2': 0.0} {'j1': 1.2619008, 'j2': 0.0, 'j3': 0.0, 'j4': 1.16715} {'p1': 0.0, 'p2': 0.0}
    0.0
torch.Size([1, 37])
(20,)


    {'source_blend': {'s1': {'j1': 1.6201598569750786, 'j2': -9.696550939104586e-08, 'j3': 0.0, 'j4': 1.2908771889594064}, 's2': {'j1': 0.9714601188898087, 'j2': 1.587307638096686, 'j3': 0.0, 'j4': 0.5545954406261444}}, 'blend_demand': {'j1': {'p1': 0.0, 'p2': 0.4476403648563108}, 'j2': {'p1': -9.607639056656139e-08, 'p2': 0.0}, 'j3': {'p1': 0.0, 'p2': 0.295

In [26]:
env.mapping_act

[(0, ['source_blend', 's1', 'j1']),
 (1, ['source_blend', 's1', 'j2']),
 (2, ['source_blend', 's1', 'j3']),
 (3, ['source_blend', 's1', 'j4']),
 (4, ['source_blend', 's2', 'j1']),
 (5, ['source_blend', 's2', 'j2']),
 (6, ['source_blend', 's2', 'j3']),
 (7, ['source_blend', 's2', 'j4']),
 (8, ['blend_blend', 'j1', 'j5']),
 (9, ['blend_blend', 'j1', 'j6']),
 (10, ['blend_blend', 'j1', 'j7']),
 (11, ['blend_blend', 'j1', 'j8']),
 (12, ['blend_blend', 'j2', 'j5']),
 (13, ['blend_blend', 'j2', 'j6']),
 (14, ['blend_blend', 'j2', 'j7']),
 (15, ['blend_blend', 'j2', 'j8']),
 (16, ['blend_blend', 'j3', 'j5']),
 (17, ['blend_blend', 'j3', 'j6']),
 (18, ['blend_blend', 'j3', 'j7']),
 (19, ['blend_blend', 'j3', 'j8']),
 (20, ['blend_blend', 'j4', 'j5']),
 (21, ['blend_blend', 'j4', 'j6']),
 (22, ['blend_blend', 'j4', 'j7']),
 (23, ['blend_blend', 'j4', 'j8']),
 (24, ['blend_demand', 'j5', 'p1']),
 (25, ['blend_demand', 'j5', 'p2']),
 (26, ['blend_demand', 'j6', 'p1']),
 (27, ['blend_demand', 'j6'